# Basic Imports

In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm

# Load traning  data

In [2]:
training_data = np.load('traning_data.npy',allow_pickle=True)

# Structure of network

In [31]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat).

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)

In [32]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


# Traning

In [33]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1  #  reserve 10% of our data for validation
val_size = int(len(X)*VAL_PCT)

train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]



In [34]:
print('Validation Size:',val_size)
print('Train Size:',len(train_X))
print('Test Size:',len(test_X))

Validation Size: 2494
Train Size: 22452
Test Size: 2494


In [37]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]
        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss}")



  0%|          | 1/225 [00:00<00:35,  6.33it/s]

torch.Size([100, 1, 50, 50]) torch.Size([100, 2])
torch.Size([100, 2])
torch.Size([100, 1, 50, 50]) torch.Size([100, 2])



  1%|          | 2/225 [00:00<00:36,  6.14it/s]

torch.Size([100, 2])
torch.Size([100, 1, 50, 50]) torch.Size([100, 2])
torch.Size([100, 2])



  2%|▏         | 4/225 [00:00<00:37,  5.92it/s]

...

torch.Size([100, 1, 50, 50]) torch.Size([100, 2])
torch.Size([100, 2])
torch.Size([100, 1, 50, 50]) torch.Size([100, 2])



 99%|█████████▉| 223/225 [00:39<00:00,  5.43it/s]

torch.Size([100, 2])
torch.Size([100, 1, 50, 50]) torch.Size([100, 2])
torch.Size([100, 2])



100%|██████████| 225/225 [00:40<00:00,  5.58it/s]

torch.Size([52, 1, 50, 50]) torch.Size([52, 2])
torch.Size([52, 2])
Epoch: 0. Loss: 0.19713585078716278


# Test accuracy

In [38]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  # returns a list, 
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))


100%|██████████| 2494/2494 [00:03<00:00, 690.76it/s]

Accuracy:  0.642


# Train on GPU

In [42]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU:", device)
else:
    device = torch.device("cpu")
    print("Running on the CPU", device)

Running on the GPU: cuda:0


In [44]:
net = Net().to(device)

In [49]:
def train(net,BATCH_SIZE = 100,    EPOCHS = 3):
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    for epoch in range(EPOCHS):
        for i in range(0, len(train_X), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
            batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+BATCH_SIZE]

            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            net.zero_grad()

            optimizer.zero_grad()   # zero the gradient buffers
            outputs = net(batch_X)
            loss = loss_function(outputs, batch_y)
            loss.backward()
            optimizer.step()    # Does the update

        print(f"Epoch: {epoch}. Loss: {loss}")


In [ ]:
train(net, EPOCHS=10)

Epoch: 0. Loss: 0.09853548556566238
Epoch: 1. Loss: 0.10401438921689987
Epoch: 2. Loss: 0.09633658826351166
Epoch: 3. Loss: 0.1109447181224823
Epoch: 4. Loss: 0.0674598291516304


# Test Accuracy

In [46]:
correct = 0
total = 0
for i in tqdm(range(0, len(test_X), BATCH_SIZE)):

    batch_X = test_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50).to(device)
    batch_y = test_y[i:i+BATCH_SIZE].to(device)
    batch_out = net(batch_X)

    out_maxes = [torch.argmax(i) for i in batch_out]
    target_maxes = [torch.argmax(i) for i in batch_y]
    for i,j in zip(out_maxes, target_maxes):
        if i == j:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))


100%|██████████| 25/25 [00:00<00:00, 53.13it/s]

Accuracy:  0.755
